In [38]:
import numpy as np
import pandas as pd

In [39]:
def create_column_filter(df):
    new_df = df.copy()
    all_columns = new_df.columns
    saved_columns = []
    
    for column_name in new_df.columns:
        if column_name == 'CLASS':
            continue
        column_value = new_df.loc[:, column_name]
        values = []
        for item in column_value:
            if pd.isna(item):
                continue
            else:
                values.append(item)
        if len(set(values)) > 1 or len(values) == 1:
            saved_columns.append(column_name)
            
    deleted_colums = list(set(all_columns) ^ set(saved_columns))
    deleted_colums.remove('CLASS')
    
    for item in deleted_colums:
        new_df.pop(item)
    return new_df, saved_columns

In [40]:
def apply_column_filter(df, column_filter):
    new_df = df.copy()
    all_columns = list(new_df.columns)
    all_columns.remove('CLASS')
    
    remained_colums = list(set(all_columns) ^ set(column_filter))
    for item in remained_colums:
        new_df.pop(item)
    return new_df

In [41]:
# Test your code (leave this part unchanged)

df = pd.DataFrame({"CLASS":[1,0,1,0,1],"A":[1,2,np.nan,4,5],"B":[1,1,1,1,np.nan],"C":["h","h",np.nan,"i","h"],"D":[np.nan,np.nan,np.nan,np.nan,np.nan]})

filtered_df, column_filter = create_column_filter(df)

new_df = pd.DataFrame({"CLASS":[1,0,0],"A":[4,5,6],"B":[1,2,1],"C":[np.nan,np.nan,np.nan],"D":[np.nan,4,5]})

filtered_new_df = apply_column_filter(new_df,column_filter)

display("df",df)
display("filtered_df",filtered_df)
display("new_df",new_df)
display("filtered_new_df",filtered_new_df)

'df'

,CLASS,A,B,C,D
0,1,1.0,1.0,h,NaN
1,0,2.0,1.0,h,NaN
2,1,NaN,1.0,NaN,NaN
3,0,4.0,1.0,i,NaN
4,1,5.0,NaN,h,NaN


'filtered_df'

,CLASS,A,C
0,1,1.0,h
1,0,2.0,h
2,1,NaN,NaN
3,0,4.0,i
4,1,5.0,h


'new_df'

,CLASS,A,B,C,D
0,1,4,1,NaN,NaN
1,0,5,2,NaN,4.0
2,0,6,1,NaN,5.0


'filtered_new_df'

,CLASS,A,C
0,1,4,NaN
1,0,5,NaN
2,0,6,NaN


In [42]:
anneal_train_df = pd.read_csv("anneal_train.csv")
anneal_test_df = pd.read_csv("anneal_test.csv")
anneal_train_df

,family,product-type,steel,carbon,hardness,temper_rolling,condition,formability,strength,non-ageing,...,s,p,shape,thick,width,len,oil,bore,packing,CLASS
0,NaN,C,A,0,0,NaN,S,2.0,0,NaN,...,NaN,NaN,SHEET,0.999,1220.0,4880,NaN,0,NaN,3
1,NaN,C,NaN,0,0,NaN,A,2.0,0,NaN,...,NaN,NaN,SHEET,0.700,1320.0,4880,NaN,0,NaN,3
2,TN,C,A,0,0,NaN,NaN,3.0,0,N,...,NaN,NaN,SHEET,0.500,1220.0,4880,NaN,0,NaN,5
3,ZS,C,A,0,50,T,NaN,NaN,0,NaN,...,NaN,NaN,SHEET,0.451,1250.0,762,NaN,0,NaN,3
4,NaN,C,A,0,85,T,NaN,NaN,0,NaN,...,NaN,NaN,COIL,4.000,1000.0,0,NaN,600,NaN,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,ZS,C,A,0,70,T,NaN,NaN,0,NaN,...,NaN,NaN,COIL,0.600,610.0,0,NaN,0,NaN,3
445,NaN,C,NaN,0,0,NaN,S,1.0,0,NaN,...,NaN,NaN,COIL,0.300,609.9,0,NaN,0,NaN,3
446,NaN,C,K,45,0,NaN,NaN,NaN,0,NaN,...,NaN,NaN,COIL,2.300,900.0,0,NaN,600,NaN,3
447,NaN,C,A,0,0,NaN,S,3.0,0,N,...,NaN,NaN,SHEET,1.000,1320.0,4880,NaN,0,NaN,3


In [43]:
anneal_train_df.columns

Index(['family', 'product-type', 'steel', 'carbon', 'hardness',
       'temper_rolling', 'condition', 'formability', 'strength', 'non-ageing',
       'surface-finish', 'surface-quality', 'enamelability', 'bc', 'bf', 'bt',
       'bw/me', 'bl', 'm', 'chrom', 'phos', 'cbond', 'marvi', 'exptl', 'ferro',
       'corr', 'blue-bright-varn-clean', 'lustre', 'jurofm', 's', 'p', 'shape',
       'thick', 'width', 'len', 'oil', 'bore', 'packing', 'CLASS'],
      dtype='object')

In [44]:
def create_imputation(dataframe):
    
    df = dataframe.copy()
    imputation = {}
    
    for col in df.columns:
        
        if col != "ID" and col != "CLASS":  
            
            if df[col].dtypes == "int" or df[col].dtype == "float":
                
                df[col].fillna(df[col].mean(),inplace=True)
                imputation[col] = (df[col].mean())
                
            else:
                df[col].fillna(df[col].mode()[0],inplace=True)
                imputation[col] = (df[col].mode()[0])
        
    return df, imputation


In [45]:
# Test your code (leave this part unchanged)

anneal_train_df = pd.read_csv("anneal_train.csv")
anneal_test_df = pd.read_csv("anneal_test.csv")
#print(anneal_train_df.head())

anneal_train_imp, imputation = create_imputation(anneal_train_df)

display(anneal_train_imp)

,family,product-type,steel,carbon,hardness,temper_rolling,condition,formability,strength,non-ageing,...,s,p,shape,thick,width,len,oil,bore,packing,CLASS
0,TN,C,A,0,0,T,S,2.000000,0,N,...,NaN,NaN,SHEET,0.999,1220.0,4880,Y,0,3.0,3
1,TN,C,A,0,0,T,A,2.000000,0,N,...,NaN,NaN,SHEET,0.700,1320.0,4880,Y,0,3.0,3
2,TN,C,A,0,0,T,S,3.000000,0,N,...,NaN,NaN,SHEET,0.500,1220.0,4880,Y,0,3.0,5
3,ZS,C,A,0,50,T,S,2.251748,0,N,...,NaN,NaN,SHEET,0.451,1250.0,762,Y,0,3.0,3
4,TN,C,A,0,85,T,S,2.251748,0,N,...,NaN,NaN,COIL,4.000,1000.0,0,Y,600,3.0,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,ZS,C,A,0,70,T,S,2.251748,0,N,...,NaN,NaN,COIL,0.600,610.0,0,Y,0,3.0,3
445,TN,C,A,0,0,T,S,1.000000,0,N,...,NaN,NaN,COIL,0.300,609.9,0,Y,0,3.0,3
446,TN,C,K,45,0,T,S,2.251748,0,N,...,NaN,NaN,COIL,2.300,900.0,0,Y,600,3.0,3
447,TN,C,A,0,0,T,S,3.000000,0,N,...,NaN,NaN,SHEET,1.000,1320.0,4880,Y,0,3.0,3


In [46]:
anneal_train_df = pd.read_csv("anneal_train.csv")
anneal_test_df = pd.read_csv("anneal_test.csv")
display(anneal_test_df["family"].mode())

anneal_train_imp, imputation = create_imputation(anneal_train_df)
display(anneal_train_imp, imputation)

0    TN
dtype: object

,family,product-type,steel,carbon,hardness,temper_rolling,condition,formability,strength,non-ageing,...,s,p,shape,thick,width,len,oil,bore,packing,CLASS
0,TN,C,A,0,0,T,S,2.000000,0,N,...,NaN,NaN,SHEET,0.999,1220.0,4880,Y,0,3.0,3
1,TN,C,A,0,0,T,A,2.000000,0,N,...,NaN,NaN,SHEET,0.700,1320.0,4880,Y,0,3.0,3
2,TN,C,A,0,0,T,S,3.000000,0,N,...,NaN,NaN,SHEET,0.500,1220.0,4880,Y,0,3.0,5
3,ZS,C,A,0,50,T,S,2.251748,0,N,...,NaN,NaN,SHEET,0.451,1250.0,762,Y,0,3.0,3
4,TN,C,A,0,85,T,S,2.251748,0,N,...,NaN,NaN,COIL,4.000,1000.0,0,Y,600,3.0,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,ZS,C,A,0,70,T,S,2.251748,0,N,...,NaN,NaN,COIL,0.600,610.0,0,Y,0,3.0,3
445,TN,C,A,0,0,T,S,1.000000,0,N,...,NaN,NaN,COIL,0.300,609.9,0,Y,0,3.0,3
446,TN,C,K,45,0,T,S,2.251748,0,N,...,NaN,NaN,COIL,2.300,900.0,0,Y,600,3.0,3
447,TN,C,A,0,0,T,S,3.000000,0,N,...,NaN,NaN,SHEET,1.000,1320.0,4880,Y,0,3.0,3


{'family': 'TN',
 'product-type': 'C',
 'steel': 'A',
 'carbon': 3.859688195991091,
 'hardness': 13.084632516703786,
 'temper_rolling': 'T',
 'condition': 'S',
 'formability': 2.251748251748252,
 'strength': 26.302895322939868,
 'non-ageing': 'N',
 'surface-finish': 'P',
 'surface-quality': 'E',
 'enamelability': 1.7142857142857142,
 'bc': 'Y',
 'bf': 'Y',
 'bt': 'Y',
 'bw/me': 'B',
 'bl': 'Y',
 'm': nan,
 'chrom': 'C',
 'phos': 'P',
 'cbond': 'Y',
 'marvi': nan,
 'exptl': nan,
 'ferro': 'Y',
 'corr': nan,
 'blue-bright-varn-clean': 'B',
 'lustre': 'Y',
 'jurofm': nan,
 's': nan,
 'p': nan,
 'shape': 'SHEET',
 'thick': 1.1911937639198218,
 'width': 769.4917594654788,
 'len': 1229.293986636971,
 'oil': 'Y',
 'bore': 35.18930957683742,
 'packing': 3.0}